This exercise is adapted from https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/


## Keras Tutorial Overview

There is not a lot of code required, but we are going to step over it slowly so that you will know how to create your own models in the future.

The steps you are going to cover in this tutorial are as follows:

1. Load Data.
2. Define Keras Model.
3. Compile Keras Model.
4. Fit Keras Model.
5. Evaluate Keras Model.
6. Tie It All Together.
7. Make Predictions

This Keras tutorial has a few requirements:

1. You have Python 2 or 3 installed and configured.
2. You have SciPy (including NumPy) installed and configured.
3. You have Keras and a backend (Theano or TensorFlow) installed and configured.

### 1. Load Data
The first step is to define the functions and classes we intend to use in this tutorial.

We will use the NumPy library to load our dataset and we will use two classes from the Keras library to define our model.

The imports required are listed below.

In [1]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense


import seaborn as sns

sns.set()


Now load the data from *file pima-indians-diabetes.data.csv* stored in Data folder (check Google Drive if unsure)

In [3]:
# load the dataset
dataset = loadtxt('.\pima-indians-diabetes.data.csv', delimiter=',')

# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

**Q:** Find out more about the dataset - such as how many columns, what kind of information does it provide etc.

In [4]:
import pandas as pd

df = pd.DataFrame(data=dataset)
# df.head()
df.describe()


,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    float64
 1   1       768 non-null    float64
 2   2       768 non-null    float64
 3   3       768 non-null    float64
 4   4       768 non-null    float64
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    float64
 8   8       768 non-null    float64
dtypes: float64(9)
memory usage: 54.1 KB


### 2. Define Keras Model
Models in Keras are defined as a sequence of layers.

We create a **Sequential model** and add layers one at a time until we are happy with our network architecture.

The first thing to get right is to ensure the input layer has the right number of input features. This can be specified when creating the first layer with the **input_dim** argument and setting it to 8 for the 8 input variables.

*Question:* How do we know the number of layers and their types?

This is a very hard question. There are heuristics that we can use and often the best network structure is found through a process of trial and error experimentation (I explain more about this here). Generally, you need a network large enough to capture the structure of the problem.

In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the Dense class. We can specify the number of neurons or nodes in the layer as the first argument, and specify the activation function using the activation argument.

We will use the rectified linear unit activation function referred to as ReLU on the first two layers and the Sigmoid function in the output layer.

It used to be the case that Sigmoid and Tanh activation functions were preferred for all layers. These days, better performance is achieved using the ReLU activation function. We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

We can piece it all together by adding each layer:

* The model expects rows of data with 8 variables (the input_dim=8 argument)
* The first hidden layer has 12 nodes and uses the relu activation function.
* The second hidden layer has 8 nodes and uses the relu activation function.
* The output layer has one node and uses the sigmoid activation function.

In [6]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))



### 3. Compile Keras Model
Now that the model is defined, we can compile it.

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to map inputs to outputs in our dataset.

We must specify the loss function to use to evaluate a set of weights, the optimizer is used to search through different weights for the network and any optional metrics we would like to collect and report during training.

In this case, we will use cross entropy as the **loss** argument. This loss is for a binary classification problems and is defined in Keras as **“binary_crossentropy“**. 

We will define the optimizer as the efficient stochastic gradient descent algorithm “adam“. This is a popular version of gradient descent because it automatically tunes itself and gives good results in a wide range of problems. 

Finally, because it is a classification problem, we will collect and report the classification accuracy, defined via the metrics argument.

In [7]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 4. Fit Keras Model
We have defined our model and compiled it ready for efficient computation.

Now it is time to execute the model on some data.

We can train or fit our model on our loaded data by calling the fit() function on the model.

Training occurs over epochs and each epoch is split into batches.

* **Epoch:** One pass through all of the rows in the training dataset.
* **Batch:** One or more samples considered by the model within an epoch before weights are updated.

One epoch is comprised of one or more batches, based on the chosen batch size and the model is fit for many epochs.

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify using the epochs argument. We must also set the number of dataset rows that are considered before the model weights are updated within each epoch, called the batch size and set using the batch_size argument.

For this problem, we will run for a small number of epochs (150) and use a relatively small batch size of 10.

These configurations can be chosen experimentally by trial and error. We want to train the model enough so that it learns a good (or good enough) mapping of rows of input data to the output classification. The model will always have some error, but the amount of error will level out after some point for a given model configuration. This is called model convergence.

In [8]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
77/77 [==============================] - 1s 1ms/step - loss: 0.9517 - accuracy: 0.6328
Epoch 2/150
77/77 [==============================] - 0s 1ms/step - loss: 0.8088 - accuracy: 0.6354
Epoch 3/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7587 - accuracy: 0.6393
Epoch 4/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7126 - accuracy: 0.6302
Epoch 5/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7092 - accuracy: 0.6367
Epoch 6/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6710 - accuracy: 0.6471
Epoch 7/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6584 - accuracy: 0.6549
Epoch 8/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6505 - accuracy: 0.6549
Epoch 9/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6450 - accuracy: 0.6510
Epoch 10/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6380 - accuracy: 0.6654
Epoch 11/

77/77 [==============================] - 0s 1ms/step - loss: 0.5484 - accuracy: 0.7161
Epoch 84/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5538 - accuracy: 0.7188
Epoch 85/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5419 - accuracy: 0.7357
Epoch 86/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5486 - accuracy: 0.7240
Epoch 87/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5403 - accuracy: 0.7357
Epoch 88/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5383 - accuracy: 0.7422
Epoch 89/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5416 - accuracy: 0.7305
Epoch 90/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5473 - accuracy: 0.7240
Epoch 91/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5433 - accuracy: 0.7305
Epoch 92/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5399 - accuracy: 0.7357
Epoch 93/150


**Q:** change the value of Epoch - what differences will you expect/see?

In [9]:
# fit the keras model on the dataset
model.fit(X, y, epochs=300, batch_size=30)

Epoch 1/300
26/26 [==============================] - 0s 2ms/step - loss: 0.5033 - accuracy: 0.7578
Epoch 2/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4928 - accuracy: 0.7708
Epoch 3/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4883 - accuracy: 0.7669
Epoch 4/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4914 - accuracy: 0.7630
Epoch 5/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4905 - accuracy: 0.7708
Epoch 6/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4882 - accuracy: 0.7656
Epoch 7/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4855 - accuracy: 0.7708
Epoch 8/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4853 - accuracy: 0.7695
Epoch 9/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4861 - accuracy: 0.7695
Epoch 10/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4882 - accuracy: 0.7734
Epoch 11/

26/26 [==============================] - 0s 2ms/step - loss: 0.4602 - accuracy: 0.7760
Epoch 84/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4681 - accuracy: 0.7721
Epoch 85/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4641 - accuracy: 0.7747
Epoch 86/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4616 - accuracy: 0.7826
Epoch 87/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4726 - accuracy: 0.7773
Epoch 88/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4625 - accuracy: 0.7891
Epoch 89/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4657 - accuracy: 0.7799
Epoch 90/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4617 - accuracy: 0.7773
Epoch 91/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4544 - accuracy: 0.7917
Epoch 92/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4530 - accuracy: 0.7956
Epoch 93/300


26/26 [==============================] - 0s 2ms/step - loss: 0.4371 - accuracy: 0.7982
Epoch 165/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4373 - accuracy: 0.8177
Epoch 166/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4387 - accuracy: 0.8086
Epoch 167/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4402 - accuracy: 0.8073
Epoch 168/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4349 - accuracy: 0.8086
Epoch 169/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4425 - accuracy: 0.8060
Epoch 170/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4490 - accuracy: 0.7917
Epoch 171/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4369 - accuracy: 0.8112
Epoch 172/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4362 - accuracy: 0.7943
Epoch 173/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4342 - accuracy: 0.8125
Epoc

26/26 [==============================] - 0s 2ms/step - loss: 0.4258 - accuracy: 0.8021
Epoch 246/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4294 - accuracy: 0.7995
Epoch 247/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4292 - accuracy: 0.8047
Epoch 248/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4208 - accuracy: 0.8112
Epoch 249/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4236 - accuracy: 0.8086
Epoch 250/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4270 - accuracy: 0.8164
Epoch 251/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4250 - accuracy: 0.8073
Epoch 252/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4297 - accuracy: 0.8034
Epoch 253/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4308 - accuracy: 0.8021
Epoch 254/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4261 - accuracy: 0.8060
Epoc

### 5. Evaluate Keras Model
We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset.

This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

You can evaluate your model on your training dataset using the **evaluate()** function on your model and pass it the same input and output used to train the model.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

The **evaluate()** function will return a list with two values. The first will be the loss of the model on the dataset and the second will be the accuracy of the model on the dataset. We are only interested in reporting the accuracy, so we will ignore the loss value.

In [10]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 1ms/step - loss: 0.4222 - accuracy: 0.8047
Accuracy: 80.47


### 7. Make Predictions
We can adapt the above example and use it to generate predictions on the training dataset, pretending it is a new dataset we have not seen before.

Making predictions is as easy as calling the predict() function on the model. We are using a sigmoid activation function on the output layer, so the predictions will be a probability in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

In [11]:
# make probability predictions with the model
predictions = model.predict(X)

# round predictions 
rounded = [round(x[0]) for x in predictions]


**Q:** now redo predictions by using the function predict_classes(). Show the outcome of the first 10 cases (code provided)

In [12]:
predictions = model.predict_classes(X)

for i in range(10):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

AttributeError: 'Sequential' object has no attribute 'predict_classes'

## Challenge 1:
Separate the training and test datasets (70% vs 30%) and rerun the evaluation

In [22]:
import numpy as np  
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 

In [23]:
model.fit(X_train, y_train, epochs=150, batch_size=10)
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))
# 74.46

Epoch 1/150
54/54 [==============================] - 0s 1ms/step - loss: 0.4072 - accuracy: 0.8063
Epoch 2/150
54/54 [==============================] - 0s 1ms/step - loss: 0.4147 - accuracy: 0.7952
Epoch 3/150
54/54 [==============================] - 0s 1ms/step - loss: 0.3827 - accuracy: 0.8082
Epoch 4/150
54/54 [==============================] - 0s 1ms/step - loss: 0.4035 - accuracy: 0.8101
Epoch 5/150
54/54 [==============================] - 0s 1ms/step - loss: 0.4025 - accuracy: 0.8101
Epoch 6/150
54/54 [==============================] - 0s 1ms/step - loss: 0.3723 - accuracy: 0.8194
Epoch 7/150
54/54 [==============================] - 0s 1ms/step - loss: 0.3849 - accuracy: 0.8119
Epoch 8/150
54/54 [==============================] - 0s 1ms/step - loss: 0.3671 - accuracy: 0.8231
Epoch 9/150
54/54 [==============================] - 0s 1ms/step - loss: 0.3680 - accuracy: 0.8250
Epoch 10/150
54/54 [==============================] - 0s 1ms/step - loss: 0.4162 - accuracy: 0.8045
Epoch 11/

54/54 [==============================] - 0s 1ms/step - loss: 0.3911 - accuracy: 0.8101
Epoch 83/150
54/54 [==============================] - 0s 1ms/step - loss: 0.3759 - accuracy: 0.8138
Epoch 84/150
54/54 [==============================] - 0s 979us/step - loss: 0.3709 - accuracy: 0.8138
Epoch 85/150
54/54 [==============================] - 0s 984us/step - loss: 0.3870 - accuracy: 0.8119
Epoch 86/150
54/54 [==============================] - 0s 926us/step - loss: 0.3700 - accuracy: 0.8194
Epoch 87/150
54/54 [==============================] - 0s 1ms/step - loss: 0.4064 - accuracy: 0.8101
Epoch 88/150
54/54 [==============================] - 0s 1ms/step - loss: 0.3675 - accuracy: 0.8212
Epoch 89/150
54/54 [==============================] - 0s 1ms/step - loss: 0.3797 - accuracy: 0.8175
Epoch 90/150
54/54 [==============================] - 0s 1ms/step - loss: 0.3635 - accuracy: 0.8250
Epoch 91/150
54/54 [==============================] - 0s 1ms/step - loss: 0.3617 - accuracy: 0.8287
Epoch 9

## Challenge 2:
How can you tune the model to improve the accuracy to be higher than what you have obtained?

In [49]:
# define the keras model
model = Sequential()
model.add(Dense(10, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
model.fit(X_train, y_train, epochs=300, batch_size=5)
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))
# 73.59 300 15
# 73.59 200 5
# 71.43 350 5
# 74.89 300 5

Epoch 1/300
108/108 [==============================] - 1s 860us/step - loss: 5.4495 - accuracy: 0.6518
Epoch 2/300
108/108 [==============================] - 0s 902us/step - loss: 2.5699 - accuracy: 0.6462
Epoch 3/300
108/108 [==============================] - 0s 995us/step - loss: 1.4818 - accuracy: 0.6015
Epoch 4/300
108/108 [==============================] - 0s 1ms/step - loss: 1.1735 - accuracy: 0.5568
Epoch 5/300
108/108 [==============================] - 0s 1ms/step - loss: 1.0734 - accuracy: 0.5512
Epoch 6/300
108/108 [==============================] - 0s 1ms/step - loss: 0.9970 - accuracy: 0.5680
Epoch 7/300
108/108 [==============================] - 0s 1ms/step - loss: 0.9432 - accuracy: 0.5866
Epoch 8/300
108/108 [==============================] - 0s 1ms/step - loss: 0.8722 - accuracy: 0.6555
Epoch 9/300
108/108 [==============================] - 0s 1ms/step - loss: 0.9084 - accuracy: 0.6611
Epoch 10/300
108/108 [==============================] - 0s 1ms/step - loss: 0.8353 - 

108/108 [==============================] - 0s 1ms/step - loss: 0.5440 - accuracy: 0.7263
Epoch 82/300
108/108 [==============================] - 0s 1ms/step - loss: 0.5345 - accuracy: 0.7281
Epoch 83/300
108/108 [==============================] - 0s 1ms/step - loss: 0.5389 - accuracy: 0.7188
Epoch 84/300
108/108 [==============================] - 0s 2ms/step - loss: 0.5312 - accuracy: 0.7467
Epoch 85/300
108/108 [==============================] - 0s 2ms/step - loss: 0.5287 - accuracy: 0.7374
Epoch 86/300
108/108 [==============================] - 0s 2ms/step - loss: 0.5372 - accuracy: 0.7430
Epoch 87/300
108/108 [==============================] - 0s 2ms/step - loss: 0.5416 - accuracy: 0.7337
Epoch 88/300
108/108 [==============================] - 0s 2ms/step - loss: 0.5259 - accuracy: 0.7393
Epoch 89/300
108/108 [==============================] - 0s 2ms/step - loss: 0.5351 - accuracy: 0.7244
Epoch 90/300
108/108 [==============================] - 0s 1ms/step - loss: 0.5285 - accuracy: 

108/108 [==============================] - 0s 2ms/step - loss: 0.5077 - accuracy: 0.7561
Epoch 161/300
108/108 [==============================] - 0s 2ms/step - loss: 0.4947 - accuracy: 0.7709
Epoch 162/300
108/108 [==============================] - 0s 2ms/step - loss: 0.4898 - accuracy: 0.7635
Epoch 163/300
108/108 [==============================] - 0s 1ms/step - loss: 0.4820 - accuracy: 0.7616
Epoch 164/300
108/108 [==============================] - 0s 2ms/step - loss: 0.4858 - accuracy: 0.7579
Epoch 165/300
108/108 [==============================] - 0s 2ms/step - loss: 0.4852 - accuracy: 0.7765
Epoch 166/300
108/108 [==============================] - 0s 1ms/step - loss: 0.4848 - accuracy: 0.7709
Epoch 167/300
108/108 [==============================] - 0s 1ms/step - loss: 0.4858 - accuracy: 0.7728
Epoch 168/300
108/108 [==============================] - 0s 2ms/step - loss: 0.4839 - accuracy: 0.7672
Epoch 169/300
108/108 [==============================] - 0s 2ms/step - loss: 0.4845 - a

108/108 [==============================] - 0s 2ms/step - loss: 0.4629 - accuracy: 0.7970
Epoch 240/300
108/108 [==============================] - 0s 2ms/step - loss: 0.4530 - accuracy: 0.7803
Epoch 241/300
108/108 [==============================] - 0s 2ms/step - loss: 0.4655 - accuracy: 0.7877
Epoch 242/300
108/108 [==============================] - 0s 1ms/step - loss: 0.4471 - accuracy: 0.7914
Epoch 243/300
108/108 [==============================] - 0s 1ms/step - loss: 0.4620 - accuracy: 0.7840
Epoch 244/300
108/108 [==============================] - 0s 1ms/step - loss: 0.4510 - accuracy: 0.7840
Epoch 245/300
108/108 [==============================] - 0s 1ms/step - loss: 0.4559 - accuracy: 0.7858
Epoch 246/300
108/108 [==============================] - 0s 2ms/step - loss: 0.4588 - accuracy: 0.7803
Epoch 247/300
108/108 [==============================] - 0s 1ms/step - loss: 0.4627 - accuracy: 0.7858
Epoch 248/300
108/108 [==============================] - 0s 2ms/step - loss: 0.4591 - a

## Challenge 3:
How can you save the model and use it again for prediction directly without (re-)training?

In [29]:


# make probability predictions with the model
predictions = model.predict(X_test)

# round predictions 
rounded = [round(x[0]) for x in predictions]


In [27]:
rounded

[1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0]